# Data preparation for UHI modeling

## Set-up

In [ ]:
# import special packages
import os
import math
import folium
import rasterio
import rioxarray as rxr
import numpy as np
import pandas as pd
import geopandas as gpd
import shapely.geometry as sg
import matplotlib.pyplot as plt

from matplotlib import pyplot
from math import radians, cos, sin, asin, sqrt
from folium.plugins import MeasureControl
from IPython.display import display
from shapely.geometry import Polygon
from shapely.geometry import box
from pyproj import CRS, Transformer

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
home_directory = os.path.expanduser( '~' )
os.chdir(home_directory + '/DS_Project/modules')

import pickle
import yaml
config_path = 'config.yml'
with open(config_path, 'r') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
path = config['data']['data'] + '/uhi_model/'
path_raw = path + 'raw/'
path_visual = path + 'visual/'
path_grid = path + 'grid/'
path_model = path + 'model/'

In [ ]:
from data_retrieval.Ecostress.utils import *
from models.UHI_modeling.disaggregate import *

## Exemplary grid and feature extraction

### Surface fractions

In [ ]:
grid_size_meters = 1000
coordinates = [11.547582, 48.114226, 11.627263, 48.155554]
polygon_gdf = create_polygon_from_coord(coordinates=coordinates)
grid = divide_polygon_into_grid(polygon_gdf.geometry[0], grid_size_meters)

In [ ]:
with open(path + 'raw/input.pkl', 'rb') as file:
    inp = pickle.load(file)
inp.to_crs(crs='EPSG:4326', inplace=True)
bbox = box(*coordinates)
inp = inp[inp.geometry.intersects(bbox)]

In [ ]:
surface_labels = inp.label.unique().tolist()
result = calculate_surface_coverage_super_fast(grid, inp, surface_labels)
surface_df = convert_dict_to_cols(result)
surface_df.head()

### Average building size

In [ ]:
with open(config['data']['city_3d_model'] + '/processed/processed_roofs.pkl', 'rb') as file:
    wind = pickle.load(file)
wind = wind[wind.geometry_4326.intersects(bbox)]

In [ ]:
height = calculate_average_height_super_fast(grid, wind)
final = gpd.GeoDataFrame(pd.merge(surface_df, height, on='id', how='inner'))
final.head(5)

In [ ]:
example_index = 5
example_id = grid.iloc[example_index, grid.shape[1]-1]
subset = grid.iloc[example_index,:].geometry
example = inp[inp.geometry.intersects(subset)]
final[final.id == example_id][['impervious','building','low vegetation','water','trees','road','avg_height']]

### Join features to grid data

In [ ]:
final = gpd.GeoDataFrame(pd.merge(grid, final, on='id', how='inner'))

In [ ]:
styles = {
    'ignore':{"fillColor": "#000000", 'color':'#000000', 'weight':0, 'fillOpacity':0.5}, 
    'impervious':{"fillColor": "#ffffff", 'color':'#ffffff', 'weight':0, 'fillOpacity':0.5}, 
    'building':{"fillColor": "#ff00ff", 'color':'#ff00ff', 'weight':0, 'fillOpacity':0.5}, 
    'low vegetation':{"fillColor": "#00ff00", '#00ff00':'pink', 'weight':0, 'fillOpacity':0.5}, 
    'water':{"fillColor": "#0000ff", 'color':'#0000ff', 'weight':0, 'fillOpacity':0.5}, 
    'trees':{"fillColor": "#008200", 'color':'#008200', 'weight':0, 'fillOpacity':0.5}, 
    'road':{"fillColor": "#ff0000", 'color':'#ff0000', 'weight':0, 'fillOpacity':0.5}, 
    'train':{"fillColor": "#505050", 'color':'#505050', 'weight':0, 'fillOpacity':0.5}, 
}
feature_example_map = folium.Map(location = [example.iloc[0,:].geometry.centroid.y, example.iloc[0,:].geometry.centroid.x], zoom_start = 14)

tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri', name = 'Esri Satellite', overlay = False, control = True).add_to(feature_example_map)

folium.GeoJson(subset.__geo_interface__, 
                style_function=lambda x: {'fillColor': 'grey', 'fillOpacity': 0.2, 'color': 'black', 'colorOpacity': 0.7},
                name = "Grid element"
                ).add_to(feature_example_map)
for label in surface_labels:
    folium.GeoJson(example[(example.label == label)].geometry, style_function=lambda x, label=label: styles[label], name=label).add_to(feature_example_map)
folium.LayerControl().add_to(feature_example_map)
feature_example_map

In [ ]:
feature_example_map.save(path_visual + 'feature_example.html')

## Visualise final results

In [ ]:
grid_size_meters = 250
with open(path + 'final_' + str(grid_size_meters) + '_d.pkl', 'rb') as file:
    final = pickle.load(file)
final.head()

In [ ]:
cols = ['impervious','building','low vegetation','water','trees','road','avg_height','nLST','wLST']
center = [(coordinates[1] + coordinates[3]) / 2, (coordinates[0] + coordinates[2]) / 2]
grid_map = folium.Map(location=center, zoom_start=13, control = True)

for idx, grids in final.iterrows():
        values = [f"{column}: {np.round(grids[column], 2)}" for column in cols]
        tooltip_text = "<br>".join(values)
        folium.GeoJson(
                grids.geometry,
                style_function = lambda x: {'fillColor': 'transparent', 'color': 'black', 'colorOpacity': 0.7},
                popup=folium.Popup(tooltip_text, max_width=300),
                name = grids['id'],
                control = False).add_to(grid_map)

tile = folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Esri', name = 'Esri Satellite', overlay = False, control = True).add_to(grid_map)

folium.LayerControl().add_to(grid_map)

grid_map

In [ ]:
grid_map.save(path_visual + 'grid_' + str(grid_size_meters) + '_d.html')